# Tutorial Implementación CART en Python


https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/

1. Índice Gini
2. Crear divisiones
3. Construir el arbol

## El índice Gini

$\sum_{c=1}^C \hat{\pi}(1-\hat{\pi}_c) = 1 - \sum_{c}\hat{\pi}^2_c$

Donde $\hat{\pi}_c = \frac{1}{|D|}\sum_{i\in D} I (y_i=c)$


* Es una funcón de costo utilizada para evaluar divisiones en el conjunto de datos
* Involucra un atributo y un valor de ese atributo. Se utiliza para dividir patrones de entrenamiento en dos grupos.
* El índice Gini nos dice que tan bien las clases estan separadas por dos grupos dada un parámetro o umbral de separación. 
* La mejor separación tendría un índice de 0 y la peor 0.5.




**Ejemplo**

Se tienen dos grupos que son divididos de alguna manera:

En el grupo 1 tenemos todos los elementos pertenecientes a la clase 0  
En el grupo 2 tenemos todos los elementos pertenecientes a la clase 1 




In [108]:
# Calcula el índice Gini para un conjunto de datos dividido.

def gini_index(groups, classes):
    
    # Contar el número de elementos que existen en groups que son los elementos pertenecientes a una división
    n_instances = float(sum([len(group) for group in groups]))
    
    # suma pesada inicializada del indice Gini para cada grupo
    gini = 0.0
    
    #por cada grupo, y cada clase ([1,0]) hacer:
    for group in groups:
        
        #calculo del numero de elementos de un grupo
        size = float(len(group))
        
        # Evitar divisiones sobre cero
        if size == 0:
            continue
        
        #inicializar score por cada grupo
        score = 0.0
        
        # anotar el score por cada clase classes = C
        for class_val in classes:
            
            #contamos la proporción de elementos perteneciente a class_val
            #p=\hat{pi}, size = |S|
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
            
        # pesar el indice del grupo por su tamaño relativo para compensar clases desbalanceadas
        gini += (1.0 - score) * (size / n_instances)
    return gini

# test Gini values
print(gini_index([[[1, 1], [1, 0]], [[1, 1], [1, 0]]], [0, 1]))
print(gini_index([[[1, 0], [1, 0]], [[1, 1], [1, 1]]], [0, 1]))

0.5
0.0


In [109]:
import matplotlib.pyplot as plt
import numpy as np

groups = [[[1, 1], [0, 0]], [[1, 1], [1, 0]],[[1,1],[1,1],[1,1]]]
classes = [0, 1]

[len(group) for group in groups]

[2, 2, 3]

In [110]:
n_instances = float(sum([len(group) for group in groups]))

In [111]:
[0,1,0,0].count(1) 

1

In [112]:
float(sum([len(group) for group in groups]))

7.0

In [113]:
row = [0,1,2]

## Crear Divisiones

Una división (*split*), esta compuesto de un atributo en el conjunto de datos y un valor *umbral* necesario para definir hasta que valor separamos los datos.

```
 |
[a1,a2,a3]
 
```
En este paso se separa el conjunto de datos en dos listas de *renglones* según un (el índice de un) atributo y una valor o umbral de división.

Crear una división involucra:

(1. Calcular el índice Gini de una división.)
2. Dividir el conjunto de datos.
3. Evaluando todos las divisiones.


Una vez que tenemos dos grupos, los evaluamos con el índice Gini.





In [114]:
### Dividiendo el conjunto de datos

In [115]:
# Dividir un conjunto de datos basado en un atributo y un valor de atributo (umbral)

def test_split(index, value, dataset):
    '''
    index: atributo seleccionado para partir
    value: umbral del atributo que divide el grupo de datos en dos
    '''
    left, right = list(), list()
    
    #index indicate the atribute of interest to split the data
    # Creamos los dos grupos de acuerdo al valor umbral *value*
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

## Evaluando todos las divisiones.

Dado un conjunto de datos debemos checar cada umbral candidato en cada atributo, evaluar el costo de cada división y encontrar la mejor división posible.

Una vez que encontramos el mejor punto de división lo usamos como nodo en nuesto arbol de decisión.

Esta estrategia se le llama algoritmo glotón, o estrategia exhaustiva.




In [116]:

# Seleccionar el mejor punto de división para un conjunto de datos
def get_split(dataset):
    
    
    #obtenemos las diferentes clases posibles que se encuentran en la última columan
    C = [row[-1] for row in dataset]
    class_values = list(set(C))
    
    #inicializamos valores
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    
    #recorremos las columnas excepto la última columna que es la etiqueta de la clase
    for index in range(len(dataset[0])-1):
        
        #por cada elemento del conjunto de datos
        for row in dataset:
            
            #dividimos en dos grupos grupos
            value = row[index]
            groups = test_split(index, value, dataset)
            
            #evaluamos esa división de acuerdo index y value
            gini = gini_index(groups, class_values)
            
            print('X%d < %.3f Gini=%.3f' % ((index+1), row[index], gini))
            
            #guardamos el indice, el atributo, y grupos asociados al menor indice gini
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

In [117]:
dataset = [
[2.77,1.78,0],
[1.72,1.16,0],
[3.67,2.81,0],
[3.96,2.61,0],
[2.99,2.20,0],
[7.49,3.16,1],
[9.00,3.33,1],
[7.44,0.47,1],
[10.12,3.23,1],
[6.64,3.31,1]
]

In [118]:
#Ejemplo: calculando el índice Gini para todas las combinaciones de umbrales $\times$ atributos

split = get_split(dataset)
print('Split: [X%d < %.3f]' % ((split['index']+1), split['value']))

X1 < 2.770 Gini=0.444
X1 < 1.720 Gini=0.500
X1 < 3.670 Gini=0.286
X1 < 3.960 Gini=0.167
X1 < 2.990 Gini=0.375
X1 < 7.490 Gini=0.286
X1 < 9.000 Gini=0.375
X1 < 7.440 Gini=0.167
X1 < 10.120 Gini=0.444
X1 < 6.640 Gini=0.000
X2 < 1.780 Gini=0.500
X2 < 1.160 Gini=0.444
X2 < 2.810 Gini=0.320
X2 < 2.610 Gini=0.417
X2 < 2.200 Gini=0.476
X2 < 3.160 Gini=0.167
X2 < 3.330 Gini=0.444
X2 < 0.470 Gini=0.500
X2 < 3.230 Gini=0.286
X2 < 3.310 Gini=0.375
Split: [X1 < 6.640]


### Nodos Terminales

Necesitamos un mecanismo para decidir cuando detener el crecimiento del ábol.

Dos estrategias son:
Definir unaprofunidad máxima 
Definir el número de elementos mínimo que el nodo puede partir para el conjunto de datos.

Árboles profundos son mas complejos y suceptibles a sobreajustar los datos de entrenamiento.

Nodos con pocos elementos de entrenamiento tienden a sobreajustarse.


In [161]:
# Funcion auxiliar para generar un nodo terminal sin hojas.
def to_terminal(group):
    '''
    Recibe dos grupos (listas) de datos de entrenamiento

    Regresa el valor de salida mas comun  en el grupo.
    '''
    
    outcomes = [row[-1] for row in group]
    print("outcomes.count: ",max(set(outcomes), key=outcomes.count))
    return max(set(outcomes), key=outcomes.count)

### División recursiva

Con las funciones 

Construir un árbol de clasificación involucra llamar la función get_split sobre los grupos generados de marea recursiva para cada nodo.

Los nodos nuevos que se agregan son llamados nodos hijos.


In [153]:
# Construye el árbol de manera recursiva mientras no se cumpla la condicion para detener el crecimiento del arbol
# 

def split(node, max_depth, min_size, depth):
    '''
    node tiene la estructura 
    
    {
    'index': 0,
    'value': 6.64,
    'groups': ([...],[...])
    }
    '''
    print("grupos")
    print(node['groups'])
    left, right = node['groups']
    del(node['groups'])
    
    # Si son grupos vacíos, hacer de nodo left y right  terminales
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    
    # Si la profunidad maxima es rebasada, detenemos el crecimiento del arbol
    # asignando los grupos de datos a nodo left y right
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    
    # si el nodo izq tiene menos elementos, detener y asignar el grupo de datos left
    # Si no dividir los datos, asignarlos al nodo left y crecer por el nodo left
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
        
    # lo mismo que el anterior
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)
        
        
    

3.3. Construyendo el Arbol

In [154]:
# Construyendo un arbol de decición 
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

In [155]:
# Print a decision tree
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

In [162]:
tree = build_tree(dataset, 1, 1)
print_tree(tree)

X1 < 2.770 Gini=0.444
X1 < 1.720 Gini=0.500
X1 < 3.670 Gini=0.286
X1 < 3.960 Gini=0.167
X1 < 2.990 Gini=0.375
X1 < 7.490 Gini=0.286
X1 < 9.000 Gini=0.375
X1 < 7.440 Gini=0.167
X1 < 10.120 Gini=0.444
X1 < 6.640 Gini=0.000
X2 < 1.780 Gini=0.500
X2 < 1.160 Gini=0.444
X2 < 2.810 Gini=0.320
X2 < 2.610 Gini=0.417
X2 < 2.200 Gini=0.476
X2 < 3.160 Gini=0.167
X2 < 3.330 Gini=0.444
X2 < 0.470 Gini=0.500
X2 < 3.230 Gini=0.286
X2 < 3.310 Gini=0.375
grupos
([[2.77, 1.78, 0], [1.72, 1.16, 0], [3.67, 2.81, 0], [3.96, 2.61, 0], [2.99, 2.2, 0]], [[7.49, 3.16, 1], [9.0, 3.33, 1], [7.44, 0.47, 1], [10.12, 3.23, 1], [6.64, 3.31, 1]])
outcomes.count:  0
outcomes.count:  1
[X1 < 6.640]
 [0]
 [1]


4. Hacer predicciones

In [143]:
a,b = ([[2.77, 1.78, 0], [1.72, 1.16, 0], [3.67, 2.81, 0], [3.96, 2.61, 0], [2.99, 2.2, 0]], [[7.49, 3.16, 1], [9.0, 3.33, 1], [7.44, 0.47, 1], [10.12, 3.23, 1], [6.64, 3.31, 1]])

In [144]:
a+b

[[2.77, 1.78, 0],
 [1.72, 1.16, 0],
 [3.67, 2.81, 0],
 [3.96, 2.61, 0],
 [2.99, 2.2, 0],
 [7.49, 3.16, 1],
 [9.0, 3.33, 1],
 [7.44, 0.47, 1],
 [10.12, 3.23, 1],
 [6.64, 3.31, 1]]

In [65]:
# Make a prediction with a decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [66]:
# Make a prediction with a decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']
 
dataset = [[2.771244718,1.784783929,0],
[1.728571309,1.169761413,0],
[3.678319846,2.81281357,0],
[3.961043357,2.61995032,0],
[2.999208922,2.209014212,0],
[7.497545867,3.162953546,1],
[9.00220326,3.339047188,1],
[7.444542326,0.476683375,1],
[10.12493903,3.234550982,1],
[6.642287351,3.319983761,1]]
 
#  predict with a stump
stump = {'index': 0, 'right': 1, 'value': 6.642287351, 'left': 0}
for row in dataset:
    prediction = predict(stump, row)
    print('Expected=%d, Got=%d' % (row[-1], prediction))

Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1


Referencias
Implementación en scikit learn

https://www.datacamp.com/community/tutorials/decision-tree-classification-python?utm_source=adwords_ppc&utm_campaignid=1455363063&utm_adgroupid=65083631748&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=&utm_creative=332602034358&utm_targetid=aud-390929969673:dsa-429603003980&utm_loc_interest_ms=&utm_loc_physical_ms=1010110&gclid=Cj0KCQjwpdqDBhCSARIsAEUJ0hO62Hlf-3oYblq8AbxroCRoAsKTYVVtFVXSycr1Ftai_vqOSzHgPb0aAvSqEALw_wcB

https://blog.quantinsti.com/gini-index/